In [1]:
import pymongo


import pytz
tzinfo = pytz.timezone('Asia/Shanghai')

myclient = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')

print(myclient.list_database_names())
mydb = myclient["a1chemy"]
tick_data = mydb['ticks']
print(mydb.list_collection_names())

['a1chemy', 'admin', 'config', 'local']
['fund', 'tags', 'ticks']


In [3]:
import a1chemy.data_source as data_source
xueqiu_client = data_source.XueQiuDataParser()
# byd_ticks = xueqiu_client.history(symbol="SZ002594", exchange='SZ' ,period = 'day', count=200, tz='Europe/London')
byd_ticks = xueqiu_client.history(symbol="SZ002594", exchange='SZ' ,period = 'day', count=200, tz='Asia/Shanghai')

print(byd_ticks)
byd_ticks.name = '比亚迪'
byd_ticks.exchange
byd_ticks.index()[199]
byd_ticks.to_dict()['ticks'][199]

{'time': Timestamp('2021-06-11 00:00:00+0800', tz='Asia/Shanghai'),
 'volume': 54008511,
 'open': 214.3,
 'high': 229.95,
 'low': 212.08,
 'close': 227.02}

In [ ]:
mongo_ticks = data_source.MongoTicks(myclient)

mongo_ticks.upsert(exchange='SZ', symbol='SZ002594', ticks=byd_ticks)

In [ ]:
mongo_ticks = data_source.MongoTicks(myclient)

t = mongo_ticks.find_one(exchange='SZ', symbol='SZ002594')
t.convert_tz(tz='Asia/Tokyo')
# t.index()[len(t.index()) - 1]
t.to_dict()['ticks'][len(t.index()) - 1]

In [24]:
import pandas as pd
mongo_ticks = data_source.MongoTicks(myclient)

t = mongo_ticks.ticks_collection.find_one({
    'exchange':'SZ',
    'symbol': 'SZ002594'
})
t['ticks'][len(t['ticks']) - 1]

index = pd.DataFrame.from_records(t['ticks'])
pd.to_datetime(index['time'][len(index)-1]).tz_localize(tz='Europe/London')

Timestamp('2021-06-10 16:00:00+0100', tz='Europe/London')

In [ ]:
import pandas as pd
import json
from a1chemy.common.ticks import Ticks
import json
result = []
for tick in tick_data.find({'symbol': 'SZ002594'}):
    t = Ticks.from_dict(tick)
    result.append(t)
    print(t.name)
    print(t.exchange)